# Load Data to Neo4j using Python

The following notebook contains the code to illustrate how to load and interact with Neo4j using Python. For the connection to the database we use the Neo4j Python Driver. Documentation can be found here: [Python Driver Neo4j](https://neo4j.com/docs/api/python-driver/current/).

We need to import the following libraries. This can be done using pip. 

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

## Set credentials for Neo4j

Credentials should be stored in a separate file: `credentials.env`

In [2]:
if os.path.exists('credentials.env'):
    load_dotenv('credentials.env', override=True)

    # Neo4j
    uri = os.getenv('NEO4J_URI')
    username = os.getenv('NEO4J_USERNAME')
    password = os.getenv('NEO4J_PASSWORD')
    database = os.getenv('NEO4J_DATABASE')
    import_directory = os.getenv('NEO4j_IMPORT')
else:
    print("File 'credentials.env' not found.")

## Setup Python Driver connection

Setup the Python Driver for Neo4j with the loaded credentials

In [3]:
class App:
    def __init__(self, uri, user, password, database=None):
        self.driver = GraphDatabase.driver(uri, auth=(user, password), database=database)
        self.database = database

    def close(self):
        self.driver.close()

    def query(self, query):
        return self.driver.execute_query(query)

    def query_params(self, query, parameters):
        return self.driver.execute_query(query, parameters_=parameters)

    def count_nodes_in_db(self):
        query = "MATCH (n) RETURN COUNT(n)"
        result = self.query(query)
        (key, value) = result.records[0].items()[0]
        return value

    def remove_nodes_relationships(self):
        query ="""
            CALL apoc.periodic.iterate(
                "MATCH (c) RETURN c",
                "WITH c DETACH DELETE c",
                {batchSize: 1000}
            )
        """
        result = self.query(query)

    def remove_all_constraints(self):
        query ="""
            CALL apoc.schema.assert({}, {})
        """
        result = self.query(query)

In [4]:
app = App(uri, username, password, database)

In [5]:
app.count_nodes_in_db()

1915

## Read data

In [6]:
flights_df = pd.read_csv('flights_sample.csv')

In [7]:
flights_df

,Unnamed: 0,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,0,2015,1,1,4,AS,98,N407AS,ANC,SEA,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2015,1,1,4,US,840,N171US,SFO,CLT,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2015,1,1,4,AS,135,N527AS,SEA,ANC,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,2015,1,7,3,EV,4992,N837AS,ATL,BQK,...,1208.0,-11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
99996,99996,2015,1,7,3,NK,394,N522NK,LAS,PHL,...,1844.0,2.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
99997,99997,2015,1,7,3,OO,5606,N582SW,SFO,BFL,...,1232.0,7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
99998,99998,2015,1,7,3,UA,1135,N38473,ORD,MCO,...,1453.0,-1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


### Airport Nodes

Create the airport nodes

In [267]:
airports_from = flights_df['ORIGIN_AIRPORT'].drop_duplicates()
airports_from.rename("name")

0            ANC
1            LAX
2            SFO
4            SEA
6            LAS
           ...  
4860107    14222
4860211    14025
4862332    13502
4864328    15497
4867209    12265
Name: name, Length: 930, dtype: object

In [268]:
airports_to = flights_df['DESTINATION_AIRPORT'].drop_duplicates()
airports_to.rename("name")

0            SEA
1            PBI
2            CLT
3            MIA
4            ANC
           ...  
4860028    14025
4861640    15497
4865073    12265
4865185    10581
4871534    10666
Name: name, Length: 930, dtype: object

In [269]:
airports_df = pd.concat([airports_from, airports_to]).drop_duplicates()
airports_df = airports_df.to_frame(name='name')

In [270]:
airports_df

,name
0,ANC
1,LAX
2,SFO
4,SEA
6,LAS
...,...
4860211,14025
4862332,13502
4864328,15497
4867209,12265


In [271]:
file_name = 'airports.csv'

In [272]:
path = import_directory + file_name

In [273]:
airports_df.to_csv(path, index=False)

### Flight relationships

Create the flight relationships

In [274]:
columns = ['YEAR', 'MONTH', 'DAY', 'AIRLINE', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']

In [275]:
columns_lowered = {x: x.lower() for x in columns}

In [276]:
flights_df = (
    flights_df[columns]
    .drop_duplicates()
    .head(100000)
)

In [277]:
flights_df = flights_df.rename(columns=columns_lowered)

In [278]:
flights_df

,year,month,day,airline,flight_number,origin_airport,destination_airport
0,2015,1,1,AS,98,ANC,SEA
1,2015,1,1,AA,2336,LAX,PBI
2,2015,1,1,US,840,SFO,CLT
3,2015,1,1,AA,258,LAX,MIA
4,2015,1,1,AS,135,SEA,ANC
...,...,...,...,...,...,...,...
99995,2015,1,7,EV,4992,ATL,BQK
99996,2015,1,7,NK,394,LAS,PHL
99997,2015,1,7,OO,5606,SFO,BFL
99998,2015,1,7,UA,1135,ORD,MCO


In [279]:
file_name = 'flights.csv'

In [280]:
path = import_directory + file_name

In [281]:
flights_df.to_csv(path, index=False)

## Load Data to Neo4j

Load the airport nodes

In [282]:
query = """
        LOAD CSV WITH HEADERS FROM "file:///airports.csv" AS row    
        MERGE (x:Airport {    name: row.name})
    """

In [283]:
app.query(query)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x384c03490>, keys=[])

Load the flight relationships

In [284]:
query = """
    LOAD CSV WITH HEADERS FROM "file:///flights.csv" AS row
    MATCH (from:Airport {name: row.origin_airport})
    MATCH (to:Airport {name: row.destination_airport})
    MERGE (from)-[x:HAS_FLIGHT {
        day: row.day, 
        airline: row.airline, 
        flight_number: row.flight_number
    }]->(to)
"""

In [285]:
app.query(query)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x139494850>, keys=[])

### Generate queries

Below there are some queries to automatically generate queries using the file itself. 

In [286]:
def generate_import_query_nodes(df, file_name, node_type):
    query = f"""
    LOAD CSV WITH HEADERS FROM "file:///{file_name}" AS row
    MERGE (x:{node_type} {{
    """
    properties = [f"{col}: row.{col}" for col in df.columns]
    query += ", ".join(properties)
    query += "})"
    query = query.replace('\n', '')
    return query

def create_constraint(node_label, property):
    query = f"CREATE CONSTRAINT {node_label}_{property} IF NOT EXISTS FOR (x:{node_label}) REQUIRE (x.{property}) IS UNIQUE"
    return query

def generate_import_query_relationships(df, file_name, relationship_type, node_from, property_from_node, property_from_df,  node_to, property_to_node, property_to_df):
    query = f"""
    LOAD CSV WITH HEADERS FROM "file:///{file_name}" AS row

    MATCH (from:{node_from} {{{property_from_node}: row.{property_from_df}}})
    MATCH (to:{node_to} {{{property_to_node}: row.{property_to_df}}})
    MERGE (from)-[x:{relationship_type} {{
    """
    properties = df.columns[2:]
    properties = [f"{col}: row.{col}" for col in properties]
    query += ", ".join(properties)
    query += "}]->(to)"
    query = query.replace('\n', '')
    return query

In [287]:
query = generate_import_query_nodes(airports_df, 'airports.csv', 'Airport')

In [288]:
print(query)

    LOAD CSV WITH HEADERS FROM "file:///airports.csv" AS row    MERGE (x:Airport {    name: row.name})


In [289]:
app.query(query)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x39288a490>, keys=[])

In [290]:
query = generate_import_query_relationships(flights_df, 'flights.csv', 'HAS_FLIGHT', 'Airport', 'name', 'origin_airport', 'Airport', 'name', 'destination_airport') 

In [291]:
query

'    LOAD CSV WITH HEADERS FROM "file:///flights.csv" AS row    MATCH (from:Airport {name: row.origin_airport})    MATCH (to:Airport {name: row.destination_airport})    MERGE (from)-[x:HAS_FLIGHT {    day: row.day, airline: row.airline, flight_number: row.flight_number, origin_airport: row.origin_airport, destination_airport: row.destination_airport}]->(to)'

In [292]:
app.query(query)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x3958e05e0>, keys=[])